In [ ]:
pip install git+https://github.com/CDCgov/phdi

In [ ]:
from pyspark.sql import SparkSession
from phdi.linkage.seed import convert_to_patient_fhir_resources

spark = SparkSession.builder.getOrCreate()

# Set up file client
account_name = "phdidevphi9d194c64"
file_system_name = "source-data"
file_path = "synthetic_patient_mpi_seed_data.parquet"
mpi_incoming_file_path = f"abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{file_path}"

def convert(mpi_incoming_file_path):
    df = spark.read.parquet(mpi_incoming_file_path)
    converted_data = {}
    for row in df.collect():
        iris_id, fhir_bundle = convert_to_patient_fhir_resources(row.asDict())
        converted_data[iris_id] = fhir_bundle
    return converted_data

converted_data = convert(mpi_incoming_file_path)
